In [1]:
# manipulacion de archivos
import os
import json
import warnings

# manipulacion y transformacion de datos
import numpy as np
print("numpy ==",np.__version__)
import pandas as pd
print("pandas ==",pd.__version__)
import geopandas as geopd
print("geopandas ==",geopd.__version__)
import pyogrio

numpy == 1.26.0
pandas == 2.0.3
geopandas == 0.9.0


In [2]:
#from google.colab import files # solo para google collab
from google.cloud import storage

# google collab
# solo se ejecuta la primera vez: cargo el json con las credenciales del api key
#object_uploaded = files.upload()

# google collab
#path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
path_api_key = '/Users/manuel/gdrive/prgrmmng/gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

In [8]:
# para entorno Google Colab o maquina local:
bucket = client_storage.get_bucket('1_transform')
data_blob = bucket.blob('gmaps_metadata_limpio.parquet')
data_gmaps_metadata = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_gmaps_metadata.info()
data_gmaps_metadata.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274220 entries, 0 to 274219
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   gmap_id          274220 non-null  object 
 1   name             274220 non-null  object 
 2   address          272234 non-null  object 
 3   description      113352 non-null  object 
 4   latitude         274220 non-null  float64
 5   longitude        274220 non-null  float64
 6   category         274220 non-null  object 
 7   avg_rating       274220 non-null  float64
 8   num_of_reviews   274220 non-null  int64  
 9   price            148480 non-null  object 
 10  hours-state_cat  274220 non-null  object 
 11  hours            240267 non-null  object 
 12  MISC             270293 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 27.2+ MB


,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat,hours,MISC
0,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours,None,{'Accessibility': ['Wheelchair accessible seat...
1,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open,"[[Tuesday, 8AM–5:30AM], [Wednesday, 8AM–5:30AM...",None


In [3]:
# para entorno Google Colab o maquina local:

os.chdir('/Users/manuel/gdrive/prgrmmng/gcloud_api_key')

prefix = 'uscensus/zcta_geo/'
dl_dir = 'zcta_geo/'

#bucket = client_storage.get_bucket('0_datoscrudos')
#blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
#for blob in blobs:
#    filename = blob.name.replace('/', '_')
#    blob.download_to_filename(dl_dir + filename)

geodata_zcta = geopd.read_file(
    'zcta_geo/uscensus_zcta_geo_tl_2020_us_zcta520.shp'
    ,engine="pyogrio"
    ).iloc[:,[1,5,9]]
geodata_zcta.rename(
    columns = {"GEOID20":"zcta5_geoid",
               "ALAND20":"zcta5_arealand"},
    inplace = True)
geodata_zcta.info()
geodata_zcta.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33791 entries, 0 to 33790
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   zcta5_geoid     33791 non-null  object  
 1   zcta5_arealand  33791 non-null  int64   
 2   geometry        33791 non-null  geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 792.1+ KB


,zcta5_geoid,zcta5_arealand,geometry
0,35592,298552385,"POLYGON ((-88.24735 33.65390, -88.24713 33.654..."
1,35616,559506992,"POLYGON ((-88.13997 34.58184, -88.13995 34.582..."


In [10]:
data_gmaps_metadata_geo = geopd.GeoDataFrame(
    data_gmaps_metadata, geometry = geopd.points_from_xy(
        data_gmaps_metadata.longitude, data_gmaps_metadata.latitude),
    crs="EPSG:4326"
    ).to_crs(geodata_zcta.crs) # type: ignore

data_gmaps_metadata_geo.info()
data_gmaps_metadata_geo.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 274220 entries, 0 to 274219
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   gmap_id          274220 non-null  object  
 1   name             274220 non-null  object  
 2   address          272234 non-null  object  
 3   description      113352 non-null  object  
 4   latitude         274220 non-null  float64 
 5   longitude        274220 non-null  float64 
 6   category         274220 non-null  object  
 7   avg_rating       274220 non-null  float64 
 8   num_of_reviews   274220 non-null  int64   
 9   price            148480 non-null  object  
 10  hours-state_cat  274220 non-null  object  
 11  hours            240267 non-null  object  
 12  MISC             270293 non-null  object  
 13  geometry         274220 non-null  geometry
dtypes: float64(3), geometry(1), int64(1), object(9)
memory usage: 29.3+ MB


,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat,hours,MISC,geometry
0,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours,None,{'Accessibility': ['Wheelchair accessible seat...,POINT (-84.57542 34.03471)
1,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open,"[[Tuesday, 8AM–5:30AM], [Wednesday, 8AM–5:30AM...",None,POINT (-80.34761 25.86948)


In [11]:
data_gmaps_metadata_zcta = geopd.sjoin(
    data_gmaps_metadata_geo,
    geodata_zcta,
    how="left"
    ,op="within"
    )
data_gmaps_metadata_zcta.info()
data_gmaps_metadata_zcta.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 274220 entries, 0 to 274219
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   gmap_id          274220 non-null  object  
 1   name             274220 non-null  object  
 2   address          272234 non-null  object  
 3   description      113352 non-null  object  
 4   latitude         274220 non-null  float64 
 5   longitude        274220 non-null  float64 
 6   category         274220 non-null  object  
 7   avg_rating       274220 non-null  float64 
 8   num_of_reviews   274220 non-null  int64   
 9   price            148480 non-null  object  
 10  hours-state_cat  274220 non-null  object  
 11  hours            240267 non-null  object  
 12  MISC             270293 non-null  object  
 13  geometry         274220 non-null  geometry
 14  index_right      273662 non-null  float64 
 15  zcta5_geoid      273662 non-null  object  
 16  zcta5_arealand   

,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat,hours,MISC,geometry,index_right,zcta5_geoid,zcta5_arealand
0,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours,None,{'Accessibility': ['Wheelchair accessible seat...,POINT (-84.57542 34.03471),17002.0,30144,57313402.0
1,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open,"[[Tuesday, 8AM–5:30AM], [Wednesday, 8AM–5:30AM...",None,POINT (-80.34761 25.86948),20897.0,33178,158223594.0


In [12]:
#### FILTRAR POR ZONA GEOGRAFICA
# filtramos las que no pertnencen al territorio EEUU
data_gmaps_metadata_zcta_usa = data_gmaps_metadata_zcta[data_gmaps_metadata_zcta["zcta5_geoid"].notnull()].copy()

# Generamos un dummy para las zonas que estan en la zona continental de EEUU
# Defino los límites geográficos de los Estados Unidos
latitud_min = 24.396308
latitud_max = 49.384358
longitud_min = -125.000000
longitud_max = -66.934570

# Filtro lugares dentro de los límites geográficos de los Estados Unidos
# Asigno el resultado a un nuevo df
data_gmaps_metadata_zcta_usa['us_continente'] = np.where((
    (data_gmaps_metadata_zcta_usa['latitude'] >= latitud_min) &
    (data_gmaps_metadata_zcta_usa['latitude'] <= latitud_max) &
    (data_gmaps_metadata_zcta_usa['longitude'] >= longitud_min) &
    (data_gmaps_metadata_zcta_usa['longitude'] <= longitud_max)),1,0)
data_gmaps_metadata_zcta_usa.info()
data_gmaps_metadata_zcta_usa.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 273662 entries, 0 to 274219
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   gmap_id          273662 non-null  object  
 1   name             273662 non-null  object  
 2   address          272180 non-null  object  
 3   description      113298 non-null  object  
 4   latitude         273662 non-null  float64 
 5   longitude        273662 non-null  float64 
 6   category         273662 non-null  object  
 7   avg_rating       273662 non-null  float64 
 8   num_of_reviews   273662 non-null  int64   
 9   price            148418 non-null  object  
 10  hours-state_cat  273662 non-null  object  
 11  hours            239887 non-null  object  
 12  MISC             269869 non-null  object  
 13  geometry         273662 non-null  geometry
 14  index_right      273662 non-null  float64 
 15  zcta5_geoid      273662 non-null  object  
 16  zcta5_arealand   

,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat,hours,MISC,geometry,index_right,zcta5_geoid,zcta5_arealand,us_continente
0,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours,None,{'Accessibility': ['Wheelchair accessible seat...,POINT (-84.57542 34.03471),17002.0,30144,57313402.0,1
1,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open,"[[Tuesday, 8AM–5:30AM], [Wednesday, 8AM–5:30AM...",None,POINT (-80.34761 25.86948),20897.0,33178,158223594.0,1


In [13]:
print(round(len(data_gmaps_metadata_zcta_usa) / len(data_gmaps_metadata_zcta) * 100,2),"%")
print(round(
    len(data_gmaps_metadata_zcta_usa.loc[
        data_gmaps_metadata_zcta_usa['us_continente'] == 1
    ]) / len(data_gmaps_metadata_zcta) * 100,2),"%")

99.8 %
98.83 %


In [7]:
data_zcta_varsxarea = pd.read_csv(
    os.path.join('uscensus_data_zcta_varsxarea.csv'),
    dtype = {'zcta5_geoid': str}).rename(columns={
        "state_usps":"state_code"}).iloc[:,[0,11,12]].drop_duplicates()
data_zcta_varsxarea.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33914 entries, 0 to 46939
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   zcta5_geoid  33914 non-null  object
 1   state_name   33914 non-null  object
 2   state_code   33914 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB
